# Import necesary packages

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib tk
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score as acc
import numpy as np
from lime import explanation
from lime import lime_base
import math
import pickle5 as pickle
from IPython.display import Image

In [2]:
from lime_timeseries import LimeTimeSeriesExplainer

# Load Dataset

In [3]:
path_to = './data/dataset.pkl'
class a_dataset(object):
    def __init__(self):
        self.X = list()
        self.Y = list()
        self.parents = list()
dataset = a_dataset()

In [4]:
with open(path_to, 'rb') as inp:
    dataset.__dict__ = pickle.load(inp)

In [5]:
dataset.X = np.array(dataset.X)
dataset.Y = np.array(dataset.Y)

In [ ]:
plt.figure()
for idx in range(len(dataset.Y)):
    if dataset.Y[idx,0] == 0:
        plt.plot(dataset.X[idx,:], 'b')
    elif dataset.Y[idx,1] == 0:
        plt.plot(dataset.X[idx,:], 'r')
plt.title('Correlations dataset')
plt.show()